In [154]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [155]:
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [156]:
pwd

'/content/drive/MyDrive/Colab Notebooks'

In [157]:
ls

'baseline.ipynb의 사본의 사본'   model-fold-0.pth   non_duplicated.csv
'ml project'                     model-fold-1.pth   non_null_data.csv
'ml project .'                   model-fold-2.pth   Untitled0.ipynb
'ml project의 사본'              model-fold-3.pth
 MNIST/                          model-fold-4.pth


## Load Data

In [158]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader
import torch.utils.data
from sklearn.model_selection import train_test_split


In [159]:
path = './non_duplicated.csv'
data = pd.read_csv(path,encoding='cp949')
data.shape
data.head(10)

,onehot(XS),onehot(S),onehot(M),onehot(L),onehot(XL),onehot(2XL),sex,height,weight,fit
0,0,0,0,0,1,0,남성,186,100,보통이에요
1,0,0,0,0,1,0,남성,182,100,보통이에요
2,0,0,0,0,0,1,남성,181,100,커요
3,0,0,0,0,1,0,남성,180,100,보통이에요
4,0,0,0,0,1,0,남성,176,100,커요
5,0,0,1,0,0,0,여성,169,100,보통이에요
6,0,0,0,0,1,0,여성,169,100,보통이에요
7,0,0,1,0,0,0,여성,169,100,보통이에요
8,0,0,1,0,0,0,여성,164,100,보통이에요
9,0,0,0,0,1,0,남성,180,105,커요


In [160]:
map_gender={'여성':1,'남성':0}

map_fit={'보통이에요':1,'커요':2,'작아요':0}



col=['sex']
data[col]=data[col].applymap(map_gender.get)

col=['fit']
data[col]=data[col].applymap(map_fit.get)

data['fit'] = data['fit'].astype('int')
data['sex'] = data['sex'].astype('int')
data['height'] = data['height'].astype('int')
data['weight'] = data['weight'].astype('int')

data['onehot(XS)'] = data['onehot(XS)'].astype('int')
data['onehot(S)'] = data['onehot(S)'].astype('int')
data['onehot(M)'] = data['onehot(M)'].astype('int')
data['onehot(L)'] = data['onehot(L)'].astype('int')
data['onehot(XL)'] = data['onehot(XL)'].astype('int')
data['onehot(2XL)'] = data['onehot(2XL)'].astype('int')

data.head(10)

,onehot(XS),onehot(S),onehot(M),onehot(L),onehot(XL),onehot(2XL),sex,height,weight,fit
0,0,0,0,0,1,0,0,186,100,1
1,0,0,0,0,1,0,0,182,100,1
2,0,0,0,0,0,1,0,181,100,2
3,0,0,0,0,1,0,0,180,100,1
4,0,0,0,0,1,0,0,176,100,2
5,0,0,1,0,0,0,1,169,100,1
6,0,0,0,0,1,0,1,169,100,1
7,0,0,1,0,0,0,1,169,100,1
8,0,0,1,0,0,0,1,164,100,1
9,0,0,0,0,1,0,0,180,105,2


In [161]:

X = data.drop(['fit'], axis=1)
y = data['fit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # test:train = 8:2

In [162]:
type(data)

pandas.core.frame.DataFrame

In [163]:
X_train=X_train.values
y_train=y_train.values
X_test=X_test.values
y_test=y_test.values

In [164]:
print(X,y)
type(X)

      onehot(XS)  onehot(S)  onehot(M)  onehot(L)  onehot(XL)  onehot(2XL)  \
0              0          0          0          0           1            0   
1              0          0          0          0           1            0   
2              0          0          0          0           0            1   
3              0          0          0          0           1            0   
4              0          0          0          0           1            0   
...          ...        ...        ...        ...         ...          ...   
2090           0          0          0          0           0            1   
2091           0          0          0          0           0            1   
2092           0          0          0          0           1            0   
2093           0          0          0          1           0            0   
2094           0          0          0          0           1            0   

      sex  height  weight  
0       0     186     100  
1      

pandas.core.frame.DataFrame

In [165]:

class Data(Dataset):
  def __init__(self, X_train, y_train):
    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    self.X = torch.from_numpy(X_train.astype(np.float32))
    # need to convert float64 to Long else 
    # will get the following error
    # RuntimeError: expected scalar type Long but found Float
    self.y = torch.from_numpy(y_train).type(torch.LongTensor)
    self.len = self.X.shape[0]
  
  def __getitem__(self, index):
    return self.X[index], self.y[index]
  def __len__(self):
    return self.len

In [166]:
traindata = Data(X_train,y_train)

In [167]:
print(traindata)

In [168]:
batch_size = 4
trainloader = DataLoader(traindata, batch_size=batch_size, 
                         shuffle=True, num_workers=2)

In [169]:
import torch.nn as nn
# number of features (len of X cols)
input_dim = 9
# number of classes (unique of y)
output_dim = 1
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.linear1 = nn.Linear(input_dim, 20)
    self.linear2 = nn.Linear(20, 10)
    self.linear3 = nn.Linear(10, 3)
  def forward(self, x):
    #x = torch.flatten(x, 1)
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = self.linear3(x)
    return x

In [170]:
clf = Network()

In [171]:
print(clf.parameters)

<bound method Module.parameters of Network(
  (linear1): Linear(in_features=9, out_features=20, bias=True)
  (linear2): Linear(in_features=20, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=3, bias=True)
)>


In [172]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(clf.parameters(), lr=0.001)

In [173]:
epochs = 10
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    # set optimizer to zero grad to remove previous epoch gradients
    optimizer.zero_grad()
    # forward propagation
    outputs = clf(inputs)
    loss = criterion(outputs, labels)
    # backward propagation
    loss.backward()
    # optimize
    optimizer.step()
    running_loss += loss.item()
  # display statistics
  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')

[1,   419] loss: 0.12378
[2,   419] loss: 0.11391
[3,   419] loss: 0.11295
[4,   419] loss: 0.11298
[5,   419] loss: 0.11426
[6,   419] loss: 0.11421
[7,   419] loss: 0.11246
[8,   419] loss: 0.11163
[9,   419] loss: 0.11285
[10,   419] loss: 0.11088


In [174]:
testdata = Data(X_test,y_train)
testloader = DataLoader(testdata, batch_size=batch_size, 
                        shuffle=True, num_workers=2)

In [175]:
dataiter = iter(testloader)
inputs, labels = dataiter.next()

In [176]:
outputs = clf(inputs)
__, predicted = torch.max(outputs, 1)

In [177]:
correct, total = 0, 0
# no need to calculate gradients during inference
with torch.no_grad():
  for data in testloader:
    inputs, labels = data
    # calculate output by running through the network
    outputs = clf(inputs)
    # get the predictions
    __, predicted = torch.max(outputs.data, 1)
    # update results
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the {len(testdata)} test data: {100 * correct // total} %')

Accuracy of the network on the 419 test data: 80 %
